In [1]:
import ROOT

In [2]:
c1 = ROOT.TCanvas()

In [3]:
ROOT.TMVA.Tools.Instance()
ROOT.TMVA.PyMethodBase.PyInitialize()
reader = ROOT.TMVA.Reader("Color:!Silent")

In [4]:
data = ROOT.TFile.Open('flixnet.root')
sig_tree = data.Get('sig_tree')

In [5]:
from array import array

branches = {}
for branch in sig_tree.GetListOfBranches():
    branchName = branch.GetName()
    branches[branchName] = array('f', [-999])
    reader.AddVariable(branchName, branches[branchName])
    
print (branches)

{'comed': array('f', [-999.0]), 'drama': array('f', [-999.0]), 'epifa': array('f', [-999.0]), 'eroti': array('f', [-999.0]), 'myste': array('f', [-999.0]), 'roman': array('f', [-999.0]), 'thril': array('f', [-999.0])}


In [6]:
for imeth in {'Fisher','LikelihoodD','MLP'} :
     reader.BookMVA( imeth, 'dataset/weights/Flixnet_Classification_'+imeth+'.weights.xml' )

                         : Booking "LikelihoodD" of type "Likelihood" from dataset/weights/Flixnet_Classification_LikelihoodD.weights.xml.
                         : Reading weight file: dataset/weights/Flixnet_Classification_LikelihoodD.weights.xml
DataSetInfo              : [Default] : Added class "Signal"
DataSetInfo              : [Default] : Added class "Background"
                         : Booked classifier "LikelihoodD" of type: "Likelihood"
                         : Booking "Fisher" of type "Fisher" from dataset/weights/Flixnet_Classification_Fisher.weights.xml.
                         : Reading weight file: dataset/weights/Flixnet_Classification_Fisher.weights.xml
                         : Booked classifier "Fisher" of type: "Fisher"
                         : Booking "MLP" of type "MLP" from dataset/weights/Flixnet_Classification_MLP.weights.xml.
                         : Reading weight file: dataset/weights/Flixnet_Classification_MLP.weights.xml
MLP                    

In [15]:
iline = 0
dictFisher, dictLD, dictMLP = {} , {} , {} 
with open('../my_python_examples/new.csv') as newseries :
    for aseries_line in newseries.readlines() :
        iline += 1
        aseries = aseries_line.strip().split(',')
        if (iline > 1):
            for branch, value in zip(sig_tree.GetListOfBranches(),range(1,8)) :
                branches[branch.GetName()][0] = float(aseries[value])
            #    print (branch.GetName(),float(aseries[value]))
            score = reader.EvaluateMVA("Fisher");  dictFisher[aseries[0]]=score
            score = reader.EvaluateMVA("LikelihoodD");  dictLD[aseries[0]]=score
            score = reader.EvaluateMVA("MLP");  dictMLP[aseries[0]]=score
            

In [16]:
dictLD

{"'Amore sulla Via della Seta'": -0.046261893156460246,
 "'Rippers - gli sbudellatori'": -0.004104664170839057,
 "'Gli elfi di Rohnblad'": -0.041312706147319994,
 "'Crazy varsity'": -0.09967079529868975,
 "'Le inchieste del commissario De Polizis'": 0.016326473933776963,
 "'Black Tiger'": -0.048121317217037114,
 "'The mind - il genio'": -0.1524746322898177}

In [17]:
sorted_Fisher = sorted(dictFisher.items(), key=lambda kv: kv[1],reverse=True)

for x,y in sorted_Fisher :
    print (f'{x:>12} : {y:.4f}') 

'The mind - il genio' : 0.9337
'Le inchieste del commissario De Polizis' : 0.3661
'Rippers - gli sbudellatori' : 0.1401
'Amore sulla Via della Seta' : 0.0625
'Black Tiger' : -0.0641
'Gli elfi di Rohnblad' : -0.1029
'Crazy varsity' : -0.1316


In [18]:
sorted_LD = sorted(dictLD.items(), key=lambda kv: kv[1],reverse=True)

for x,y in sorted_LD :
    print (f'{x:>12} : {y:.4f}') 

'Le inchieste del commissario De Polizis' : 0.0163
'Rippers - gli sbudellatori' : -0.0041
'Gli elfi di Rohnblad' : -0.0413
'Amore sulla Via della Seta' : -0.0463
'Black Tiger' : -0.0481
'Crazy varsity' : -0.0997
'The mind - il genio' : -0.1525


In [19]:
sorted_MLP = sorted(dictMLP.items(), key=lambda kv: kv[1],reverse=True)

for x,y in sorted_MLP :
    print (f'{x:>12} : {y:.4f}') 

'The mind - il genio' : 0.9510
'Rippers - gli sbudellatori' : 0.9325
'Amore sulla Via della Seta' : 0.8094
'Crazy varsity' : 0.5029
'Black Tiger' : 0.3864
'Le inchieste del commissario De Polizis' : 0.1424
'Gli elfi di Rohnblad' : 0.0011
